In [32]:
%load_ext autoreload
%autoreload 2
from channel_utils import Channel
from encoder_utils import Encoder
from decoder_utils import Decoder
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
L = 10 # Number of previous bits used
N = 20 # Number of output bits per input bit

In [34]:
def generate_generator_matrix():
    """
    Creates the generator matrix.
    We make sure that for each generator_polynomial (A line of the generator matrix), the first and the last bits are set to 1
    This is because we at least want b_i and b_{i-L} to be used to use the whole range L. The remaining coefficients are chosen
    at random as no method exists to optimally pick them deterministically.
    """
    generator_matrix = np.random.choice([0, 1], size=(N, L+1))
    generator_matrix[:,0] = 1
    generator_matrix[:,L] = 1
    return generator_matrix
generator_matrix = generate_generator_matrix()
print(generator_matrix)

[[1 1 0 0 0 1 0 0 1 0 1]
 [1 0 0 1 0 0 1 1 1 1 1]
 [1 1 1 0 1 1 1 0 0 1 1]
 [1 1 1 1 1 0 0 1 0 0 1]
 [1 1 0 1 1 1 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 1 0 1]
 [1 0 0 1 1 0 0 1 0 1 1]
 [1 0 1 0 0 0 0 0 1 0 1]
 [1 1 0 1 1 0 1 0 1 1 1]
 [1 0 0 1 0 1 0 0 1 0 1]
 [1 0 1 0 0 0 1 0 1 1 1]
 [1 0 0 1 0 0 0 1 0 1 1]
 [1 1 1 1 0 1 0 0 0 1 1]
 [1 1 1 0 1 1 1 0 1 1 1]
 [1 0 0 1 0 1 1 0 1 0 1]
 [1 1 0 0 0 0 0 1 0 0 1]
 [1 1 1 1 0 0 1 0 0 0 1]
 [1 0 0 0 1 1 0 1 0 0 1]
 [1 1 1 1 0 1 1 1 0 0 1]
 [1 0 0 1 0 0 0 1 0 0 1]]


In [35]:
channel = Channel(changingIndex=True, verbose=False)
encoder = Encoder(input_file='80char.txt', output_file='encoded.txt', generator_matrix=generator_matrix, verbose=False)
decoder = Decoder(input_file='noisy.txt', output_file='decoded.txt', generator_matrix=generator_matrix, verbose=False)

In [36]:
encoder.encode()

Original file : b'Hello world, planet, solar system all the way to A-Centuri and the entire galaxy'

Bits array : [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 

In [37]:
channel.sendFileWithOutput(inputFileName='encoded.txt', outputFileName='noisy.txt')

Channel sending with H value 1 :
[-1.  0. -1. ...  0.  1.  1.]


In [ ]:
decoder.decode()

Compact_values_length : 12800
nb_trellis_sections : 640
Trellis height : 1024
Trellis width  : 641
Decoding ...
